# Set up the CNN + model, Train it and Test it

##### Before running this script, make sure that your Google Drive folder contains the tiles you created (`step 1`) and the 5 CSVs that you created (`step 3`) to describe the 3 data subsets (1 CSV file), the annotations for each (3 CSV files) and the class list (1 CSV file).

<a href="https://colab.research.google.com/github/gl7176/GreySealCNN/blob/master/4_CNN_setup_training_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
#####  <center> Be sure to update this hyperlink above if you clone and want to point to a different GitHub </center>

### Connect to our Google Drive folder and pull all data
Note: when you run this it will give you a link that you must click. You must give Google some permissions, then copy a code into a box that comes up in the output section of this code.

In [2]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
#    The mess of characters before "in parents" is the address of the google drive folder getting pulled
#    (when you open the folder in google drive on a browser, it is the mess of characters at the end of the URL)
#    for example, it currently points to: https://drive.google.com/drive/folders/1INuRNVKvKMy8L_Nb6lmoVbyvScWK0-0D

file_list = drive.ListFile(
    {'q': "'1INuRNVKvKMy8L_Nb6lmoVbyvScWK0-0D' in parents"}).GetList()

#    this bit pulls every file in the directory specified above
count = 0
for f in file_list:
  count += 1
  if count % 10 == 0:
    print(count)
  # 3. Create & download by id.
  fname = os.path.join(local_download_path, f['title'])
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)
  

ModuleNotFoundError: No module named 'google.colab'

### Install the Convolutional Neural Network that will do the detections. 

This section sets up the software and pulls code for a CNN model called "RetinaNet" which uses the model "ResNet-50" as a subcomponent. This section then loads data for an existing ResNet-50 model (pre-trained for object detection) which we will further train for our task.

In [ ]:
# clear any existing builds and install the versions that work for us
! pip uninstall --yes keras
! pip install keras==2.4
! pip uninstall --yes tensorflow
! pip install tensorflow==2.3.0

In [ ]:
# copy the files for RetinaNet
# note that this build is now deprecated, but we are fine with that
! git clone https://github.com/fizyr/keras-retinanet/

In [ ]:
# change directory and install RetinaNet from the copied code
% cd keras-retinanet

! pip install .

In [ ]:
! python setup.py build_ext --inplace

In [ ]:
% cd ../

# get the pre-trained ResNet-50 model
! wget -P data "https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5"

### Train the Model

We're giving our model the pre-trained weights that we downloaded above, and then we're telling it to use the training data in annotations_train.csv and to run for `10 epochs` each with `20 steps` with a `batch-size of 2`. An epoch is a group of steps after which the model calculates its accuracy; a step is an increment of training the model on one batch or subset of files.

For real applications, we're going to want to step up the epoch and step counts.

In [ ]:
! keras-retinanet/keras_retinanet/bin/train.py \
--weights data/resnet50_coco_best_v2.1.0.h5 \
--epochs 10 --steps 20 --batch-size 2 \
csv data/annotations_train.csv data/classes.csv \
--val-annotations data/annotations_valid.csv

This next section converts the model from training mode to inference mode so it can be used to detection seals. This conversion process take a little time.

In [ ]:
# note that we are naming our model "test_model" and locating it in the "snapshots" directory. Customize if wanted
! keras-retinanet/keras_retinanet/bin/convert_model.py snapshots/resnet50_csv_10.h5 snapshots/test_model.h5

### Run Detection in inference mode

This section sets up the environment, importing modules for python tasks and specific to keras+retinanet

In [ ]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import json
from random import shuffle

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.compat.v1.ConfigProto()    
    config.gpu_options.allow_growth = True
    return tf.compat.v1.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
tf.compat.v1.keras.backend.set_session(get_session())

### Load RetinaNet model

Now we will load the model that you just converted into inference mode: it is called `test_model.h5`.

In [ ]:
model_path = 'snapshots/test_model.h5'

print(model_path)

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# you can check out a model summary by uncommenting this line
#print(model.summary())

# load label to names mapping for visualization purposes
# pull labels from classes.csv
import csv
with open(local_download_path + '/classes.csv', "r") as f:
    reader = csv.reader(f, delimiter=",")
    labels_to_names = {int(i[1]):i[0] for i in reader}

### Load test imagery

Now we will load the "test" subset of images that we downloaded into our data directory during setup (as listed in the subsets CSV file). Just to check, we'll print out the first five names of those images.

In [38]:
# load imagery
image_dir = "data/"

# this code pulls only files from the test or validation subset
# as specified in this variable, "target_subset" either "test" or "validation"
target_subset = "test"

image_list = []
with open(local_download_path + '/subset_list.csv', "r") as f:
    reader = csv.reader(f, delimiter=",")
    for i in reader:
      if i[1] == target_subset:
        image_list.append(image_dir + i[0])
print(image_list[:5])


['data/2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---159.png', 'data/2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---193.png', 'data/2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---27.png', 'data/2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---212.png', 'data/2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---94.png']


### Test out detections

Now we'll visualize some detections from our model to see how it performs. Each detection has a "confidence score" that describes the CNN's confidence that the detection is correct. Change the minimum confidence score (the first line of code) and re-run the code to check out how your "confidence threshold" affects the numbers of false positives and false negatives.

In [ ]:
min_score = 0.5 # this is the CNN's confidence that the detection is correct
detection_iterations = 10 # max number of images to visualize

visualize = True

detections = {}

total_time = 0

count = 0
shuffle(image_list)

for image_path in image_list:
    if count > detection_iterations:
        break
    else:
        count +=1
        
    image = read_image_bgr(image_path)

    if visualize:
        # copy to draw on
        draw = image.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start

    # correct for image scale
    boxes /= scale
    if any(score >= min_score for score in scores[0]):
        detections[image_path] = []

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < min_score:
            break

        #print(score)
        #print(box)

        # TODO this does create a slight error in the boxes, might be worth doing something like
        # list(map(str, box) but then would need to cast on the other end back to float
        b = box.astype(int)
        detections[image_path].append({"box" : b, "label" : label, "score" : score})

        if visualize:
            color = label_color(label)

            # b = box.astype(int)
            draw_box(draw, b, color=color)

            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)

    if any(score >= min_score for score in scores[0]):
        if visualize:
            plt.figure(figsize=(10, 10))
            plt.axis('off')
            plt.imshow(draw)
            plt.show()
    
print("Finished, time per image:", total_time/len(image_list))

### Run Detections on all tiles
This section repeats the process we just tested for all tiles that make up our orthomosaic. If you want to experiment, you can vary the confidence threshold and the amount of time the model trains, then look at how it affects the resulting detections.


In [ ]:
visualize = False
min_score = 0.5 # this is the CNN's confidence that the detection is correct

detections = {}

total_time = 0

for image_path in image_list:
       
    image = read_image_bgr(image_path)

    if visualize:
        # copy to draw on
        draw = image.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start

    # correct for image scale
    boxes /= scale
    if any(score >= min_score for score in scores[0]):
        detections[image_path] = []

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < min_score:
            break

        #print(score)
        #print(box)

        # TODO this does create a slight error in the boxes, might be worth doing something like
        # list(map(str, box) but then would need to cast on the other end back to float
        b = box.astype(int)
        detections[image_path].append({"box" : b, "label" : label, "score" : score})

        if visualize:
            color = label_color(label)

            # b = box.astype(int)
            draw_box(draw, b, color=color)

            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)

    if any(score >= min_score for score in scores[0]):
        if visualize:
            plt.figure(figsize=(10, 10))
            plt.axis('off')
            plt.imshow(draw)
            plt.show()
    
print("Finished, time per image:", total_time/len(image_list))

Run an evaluation script to get the mean average precision (mAP) of the CNN. 

mAP is a model evaluation metric that is relative (aka it can be challenging to compare mAP values across datasets), but a great general metric for different models and approaches to detection objects on the same dataset. 

Read more about mAP here: https://tarangshah.com/blog/2018-01-27/what-is-map-understanding-the-statistic-of-choice-for-comparing-object-detection-models/

In [1]:
! keras-retinanet/keras_retinanet/bin/evaluate.py csv data/annotations_test.csv data/classes.csv snapshots/test_model.h5

'keras-retinanet' is not recognized as an internal or external command,
operable program or batch file.


### Export detections##
Write out the detections to a json file that can be used in a GIS for  spatial databases and/or visualizations.

In [ ]:
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [ ]:
with open('data/new_detections.json', 'w') as fp:
    json.dump(detections, fp, cls=MyEncoder)

In [ ]:
from google.colab import files
files.download("/content/data/new_detections.json")

### Export model (if wanted)

In [ ]:
# note that this is generally only necessary if we are planning to shop the model around to different datasets
files.download("/content/snapshots/test_model.h5")

#### At the end of this script you should have a single JSON file downloaded. Drop this in the Google Drive folder so it can be exported in the next step.

Next steps:

5) export CNN outputs